In [17]:
import mysql.connector as sql
import pandas as pd
from mysql.connector import Error

In [18]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = sql.connect(host=host_name, user=user_name, passwd=user_password)
        print("MySQL Database connection successful")
    except Error as err:
        print(f"The error '{err}' occurred")

    return connection

pw = "Pratham123***"
connection = create_server_connection("localhost", "root", pw)

MySQL Database connection successful


In [19]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = sql.connect(host=host_name, user=user_name, passwd=user_password, database=db_name)
        print("MySQL Database connection successful")
    except Error as err:
        print(f"The error '{err}' occurred")

    return connection

connection = create_db_connection("localhost", "root", pw, "healthcare_db")

MySQL Database connection successful


In [20]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"The error '{err}' occurred")

In [21]:
def run_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        columns = [column[0] for column in cursor.description]
        df = pd.DataFrame(result, columns=columns)
        return df
    except Error as err:
        print(f"The error '{err}' occurred")

In [22]:
# execute_query(connection, "DROP DATABASE IF EXISTS healthcare")

## Problem Statement 26:

In [23]:
execute_query(connection,"""
            CREATE TABLE IF NOT EXISTS hospital_log (
    district VARCHAR(255),
    location VARCHAR(10),
    operation VARCHAR(10),
    operation_date DATE,
    is_government_hospital BOOLEAN
);""")

Query successful


In [24]:
execute_query(connection,"""
              DROP TRIGGER IF EXISTS all_hospitals_log_insert;              
              """)

Query successful


In [25]:
execute_query(connection,"""
              CREATE TRIGGER all_hospitals_log_insert
AFTER INSERT ON all_hospitals
FOR EACH ROW
INSERT INTO hospital_log (district, location, operation, operation_date, is_government_hospital)
VALUES (NEW.`State/UT`, 'Urban', 'INSERT', CURRENT_DATE, FALSE);""")

Query successful


In [26]:
execute_query(connection,"""DROP TRIGGER IF EXISTS all_hospitals_log_delete;""")

Query successful


In [27]:
execute_query(connection,"""
              CREATE TRIGGER all_hospitals_log_delete
AFTER DELETE ON all_hospitals
FOR EACH ROW
INSERT INTO hospital_log (district, location, operation, operation_date, is_government_hospital)
VALUES (OLD.`State/UT`, 'Urban', 'DELETE', CURRENT_DATE, FALSE);""")

Query successful


In [28]:
run_query(connection, "show triggers;")

,Trigger,Event,Table,Statement,Timing,Created,sql_mode,Definer,character_set_client,collation_connection,Database Collation
0,all_hospitals_log_insert,INSERT,all_hospitals,"INSERT INTO hospital_log (district, location, ...",AFTER,2024-03-13 03:34:34.180,"{ERROR_FOR_DIVISION_BY_ZERO, ONLY_FULL_GROUP_B...",root@localhost,utf8mb4,utf8mb4_0900_ai_ci,utf8mb4_0900_ai_ci
1,all_hospitals_log_delete,DELETE,all_hospitals,"INSERT INTO hospital_log (district, location, ...",AFTER,2024-03-13 03:34:34.200,"{ERROR_FOR_DIVISION_BY_ZERO, ONLY_FULL_GROUP_B...",root@localhost,utf8mb4,utf8mb4_0900_ai_ci,utf8mb4_0900_ai_ci


## Problem Statement 27

In [30]:
execute_query(connection, """
            CREATE TABLE IF NOT EXISTS hospital_bed_log (
    district VARCHAR(100),
    location VARCHAR(10),
    operation_date DATE,
    beds_changed INTEGER,
    operation VARCHAR(10),
    is_government_hospital BOOLEAN
        );""")

Query successful
